In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [104]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [105]:
feed_temp_ins4 = log_rawDF.query('Time >= 1635246000 and Time <= 1641567595 and Unit =="FPC24" and Type == 2 and Parameter == "Reactor"')

In [106]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Recipe', regex=False, case=False, na=False)]

In [107]:
programID_DF.head(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350408,2021-11-2 8:37:25,1635838645,243296,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.0857142857142857
350510,2021-11-2 10:57:24,1635847044,749185,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.514285714285714
350657,2021-11-2 11:33:29,1635849209,742078,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.257142857142857
352123,2021-11-3 8:33:30,1635924810,738753,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.114285714285714
352176,2021-11-3 10:17:39,1635931059,739134,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.685714285714286
352234,2021-11-3 10:29:26,1635931766,741717,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.342857142857143
352821,2021-11-3 15:49:45,1635950985,246696,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.142857142857143
352842,2021-11-3 16:8:45,1635952125,738636,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.857142857142857
352891,2021-11-3 16:20:18,1635952818,238720,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.428571428571429
354615,2021-11-5 10:51:40,1636105900,248642,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.142857142857143


In [109]:
new_DF = log_rawDF.copy()

In [110]:
new_DF_ins4 = new_DF.query('Time >= 1635246000 and Time <= 1641567595')

In [111]:
new_ins4 = new_DF_ins4[new_DF_ins4['Message'].str.contains('FPC24', regex=False, case=False, na=False)]
#programID_DF = programID_DF[programID_DF['Value2'].str.contains('FPC24', regex=False, case=False, na=False)]

In [113]:
new_ins4.tail(2)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,"Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.10228</td><td width=""12.5%"">level</td><td width=""12.5%"">1.15110</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,"Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.08463</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04373</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",1.14025,1.14535


In [117]:
no_html_DF =  new_ins4.copy()

In [118]:
#no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<[^<>]*>', '', regex=True)
no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)

In [119]:
no_html_DF.tail()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
446599,2021-12-29 16:23:29,1640791409,196243,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose)DB dose0.04420DB level0.00000level0.04361,0.22822,0.21332
446622,2021-12-29 16:30:7,1640791807,667000,2,FFU,T10,Dose started for NaOH_0.1M_rinse to Tank = FPC24_Reactor (target dose and current real dose)DB total dose0.00000DB level0.91182level0.94773,0.91429,0.00000
446634,2021-12-29 16:40:9,1640792409,187728,2,FFU,T10,Dose ended at low level for NaOH_0.1M_rinse to Tank = FPC24_Reactor (DB total dose and current real dose)DB dose0.05251DB level0.00000level0.04559,0.91182,0.90214
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose)DB total dose0.00000DB level1.10228level1.15110,1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose)DB dose0.08463DB level0.00000level0.04373,1.14025,1.14535
